In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Legacy Chains

In [2]:
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

legacy_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=prompt)

legacy_result = legacy_chain({"adjective": "funny"})
legacy_result

/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_1918/3920434221.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  legacy_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=prompt)
/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_1918/3920434221.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  legacy_result = legacy_chain({"adjective": "funny"})


{'adjective': 'funny',
 'text': 'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'}

### LCEL 

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = prompt | ChatOpenAI(model="gpt-4o-mini")| StrOutputParser()

chain.invoke({"adjective": "funny"})

"Why don't scientists trust atoms? \n\nBecause they make up everything!"

### Legacy RAG

In [4]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma


embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
llm = ChatOpenAI(model="gpt-4o-mini")
retriever = db.as_retriever()

In [5]:
from langchain import hub
from langchain.chains import RetrievalQA

prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_llm(llm, retriever=retriever, prompt=prompt) # RetrievalQA is already deprecated

qa_chain("What does the cat like to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'What does the cat like to eat?',
 'result': 'The cat likes to eat lasagna.'}

### LCEL

In [6]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("What does the cat like to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'The cat likes to eat lasagna.'